# Домашнее задание (пояснение)

In [ ]:
!wget https://storage.yandexcloud.net/natasha-nerus/data/nerus_lenta.conllu.gz

--2021-03-12 09:49:03--  https://storage.yandexcloud.net/natasha-nerus/data/nerus_lenta.conllu.gz
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243, 2a02:6b8::1d9
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1961465886 (1.8G) [application/octet-stream]
Saving to: ‘nerus_lenta.conllu.gz’

nerus_lenta.conllu. 100%[===================>]   1.83G  21.0MB/s    in 92s     

2021-03-12 09:50:36 (20.4 MB/s) - ‘nerus_lenta.conllu.gz’ saved [1961465886/1961465886]



In [ ]:
!pip install nerus

In [ ]:
from nerus import load_nerus
from tqdm.notebook import tqdm

In [ ]:
docs = load_nerus("nerus_lenta.conllu.gz")

In [ ]:
list_of_sent = []
list_of_tags = []
for doc in tqdm(docs):
  for sent in doc.sents:
    list_of_sent_toks = []
    list_of_sent_tags = []
    for tok in sent.tokens:
      list_of_sent_toks.append(tok.text)
      list_of_sent_tags.append(tok.pos)
    list_of_sent.append(list_of_sent_toks)
    list_of_tags.append(list_of_sent_tags)

In [15]:
word_to_ind = dict()
pos_to_ind = dict()

word_to_ind['<PAD>'] = 0
word_to_ind['<UNK>'] = 1

for sent in list_of_sent[:-100]:
  for word in sent:
    if word not in word_to_ind:
      word_to_ind[word] = len(word_to_ind)

pos_to_ind['<PAD>'] = 0
for sent in list_of_tags[:-100]:
  for tag in sent:
    if tag not in pos_to_ind:
      pos_to_ind[tag] = len(pos_to_ind)

NameError: ignored

In [ ]:
len(word_to_ind)

64842

In [ ]:
X_train_data = []
X_test_data = []
Y_train_data = []
Y_test_data = []

for sent in list_of_sent[:-100]:
  x_data = []
  for word in sent:
    x_data.append(word_to_ind.get(word, 0))
  X_train_data.append(x_data)

for sent in list_of_sent[-100:]:
  x_data = []
  for word in sent:
    x_data.append(word_to_ind.get(word, 0))
  X_test_data.append(x_data)

for sent in list_of_tags[:-100]:
  x_data = []
  for word in sent:
    x_data.append(pos_to_ind.get(word, 0))
  Y_train_data.append(x_data)

for sent in list_of_tags[-100:]:
  x_data = []
  for word in sent:
    x_data.append(pos_to_ind.get(word, 0))
  Y_test_data.append(x_data)

In [ ]:
len(X_train_data), len(X_test_data), len(Y_train_data), len(Y_test_data)

(27606, 100, 27606, 100)

In [ ]:
import numpy as np

In [ ]:
x_cur_batch = X_train_data[:8]
y_cur_batch = Y_train_data[:8]


In [ ]:
x_batch = np.zeros(shape=(len(x_cur_batch), max(list(map(len, x_cur_batch)))), )
y_batch = np.zeros(shape=(len(y_cur_batch), max(list(map(len, y_cur_batch)))), )

In [ ]:
x_batch.shape, y_batch.shape

((8, 26), (8, 26))

In [ ]:
for i, sent in enumerate(x_cur_batch):
  x_batch[i, :len(sent)] = sent

for i, sent in enumerate(y_cur_batch):
  y_batch[i, :len(sent)] = sent

In [4]:
import torch

In [ ]:
x_batch_tr = torch.from_numpy(x_batch).long()
y_batch_tr = torch.from_numpy(y_batch).long()

In [ ]:
x_batch_tr

tensor([[  2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,  14,  15,
          16,  17,  18,  19,   9,  20,  21,  22,  23,   0,   0,   0],
        [ 24,  25,  26,   9,  27,  28,  29,  30,  31,  32,  33,  10,  34,   9,
          35,   9,  36,  37,  38,  39,   9,  40,  41,  10,  42,  23],
        [  2,  43,   9,  44,  45,  46,  47,  10,  13,  48,  49,  37,  50,  51,
          52,  23,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [ 53,  54,  55,  56,  57,   9,  44,  17,  18,  58,  59,  60,  61,  62,
          63,  64,  65,  55,  23,   0,   0,   0,   0,   0,   0,   0],
        [ 24,  66,  67,   9,  10,  68,  69,  18,  19,  70,  71,  72,  73,  23,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [ 74,  75,  76,  77,  78,   9,  79,  80,  81,  23,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [ 82,  83,  84,  85,  86,  87,  88,  89,  90,  91,  92,  23,   0,   0,
           0,   0,   0,   0

In [ ]:
y_batch_tr

tensor([[ 1,  2,  3,  1,  4,  4,  5,  6,  2,  7,  1,  4,  5,  8,  3,  1,  2,  1,
          6,  5,  4,  4,  6,  0,  0,  0],
        [ 2,  1,  4,  6,  8,  9,  3,  1,  5,  1,  1,  2,  3,  6,  3,  6,  3, 10,
          3,  1,  6, 10,  8,  2,  4,  6],
        [ 1,  5,  6, 11,  3,  1,  1,  2,  4,  6,  1, 10,  1,  1,  1,  6,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0],
        [ 2,  1,  1,  5,  3,  6, 11,  1,  2,  3,  1,  2,  1,  5,  8,  2, 12,  1,
          6,  0,  0,  0,  0,  0,  0,  0],
        [ 2,  1,  4,  6,  2,  3,  1,  2,  1,  5, 12,  1,  1,  6,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0],
        [ 9,  2, 12,  1, 12,  6, 11,  1,  8,  6,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0],
        [ 3,  3,  1, 13,  5,  1,  1,  3,  1,  3,  1,  6,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0],
        [ 2,  9,  5,  1,  4,  2,  4,  4,  4,  2,  1,  1,  5,  1,  1,  2,  1,  1,
          1,  6,  5,  4,  6,  0,  0,  0]])

In [ ]:
class Encoder(torch.nn.Module):
    @property
    def device(self):
        return next(self.parameters()).device
        
    def __init__(self,
                 vocab_dim,
                 emb_dim = 10, 
                 hidden_dim = 10,
                 num_layers = 3,
                 output_dim = len(pos_to_ind),
                 bidirectional = False):
        super(Encoder, self).__init__()
        
        self.num_direction = int(bidirectional + 1)
        self.emb_dim = emb_dim
        self.hidden_dim = hidden_dim

        self.embedding = torch.nn.Embedding(vocab_dim, emb_dim, padding_idx=0)

        self.encoder = torch.nn.LSTM(
            emb_dim, hidden_dim, num_layers, bidirectional = bidirectional, batch_first=True)
        
        self.linear = torch.nn.Linear(10, output_dim)
    def forward(self, input):
        input = self.embedding(input)
        # input = torch.transpose(input, 0, 1)
        d, _ = self.encoder(input)
        # input = torch.transpose(input, 0, 1)
        return self.linear(d)

In [ ]:
model = Encoder(vocab_dim = len(word_to_ind))

In [ ]:
loss_function = torch.nn.CrossEntropyLoss(ignore_index=0)

In [ ]:
pred = model(x_batch_tr)

In [ ]:
pred.shape

torch.Size([8, 26, 18])

In [ ]:
loss_function(pred.transpose(1, 2), y_batch_tr)

tensor(3.0964, grad_fn=<NllLoss2DBackward>)

# Task 3

In [ ]:
dataset = torch.utils.data.TensorDataset(x_batch_tr, x_batch_tr)

In [ ]:
x_batch_tr.shape, x_batch_tr.shape

(torch.Size([8, 26]), torch.Size([8, 26]))

In [ ]:
for x_b, y_b in torch.utils.data.DataLoader(dataset, batch_size=2):
  pass

In [ ]:
x_b.shape

torch.Size([2, 26])

In [ ]:
y_b.shape

torch.Size([2, 26])

In [ ]:
class Encoder(torch.nn.Module):
    @property
    def device(self):
        return next(self.parameters()).device
        
    def __init__(self,
                 vocab_dim,
                 emb_dim = 10, 
                 hidden_dim = 10,
                 num_layers = 3,
                 output_dim = len(pos_to_ind),
                 bidirectional = False):
        super(Encoder, self).__init__()
        
        self.num_direction = int(bidirectional + 1)
        self.emb_dim = emb_dim
        self.hidden_dim = hidden_dim

        self.embedding = torch.nn.Embedding(vocab_dim, emb_dim, padding_idx=0)

        self.encoder = torch.nn.LSTM(
            emb_dim, hidden_dim, num_layers, bidirectional = bidirectional, batch_first=True)
        
        self.linear = torch.nn.Linear(hidden_dim+hidden_dim, 1000)
    def forward(self, input):
        input = self.embedding(input)
        _, (h, c) = self.encoder(input)

        act = torch.cat([h, c], dim=-1).transpose(0, 1)[:, -1, :]
        return self.linear(act)
        # act = act.reshape(len(input), -1)
        # return self.linear(d)

Немного добавил к семинару

In [44]:
class Decoder(torch.nn.Module):
    @property
    def device(self):
        return next(self.parameters()).device
    def __init__(self, vocab_dim, 
                 emb_dim=128, 
                 hidden_dim=256, 
                 latent_dim=1000):
        super(Decoder, self).__init__()
        

        self.latent_to_h0 = torch.nn.Linear(latent_dim, hidden_dim)
        self.latent_to_c0 = torch.nn.Linear(latent_dim, hidden_dim)
        
        self.emb = torch.nn.Embedding(vocab_dim, emb_dim)

        self.lstm = torch.nn.LSTM(emb_dim, hidden_dim, batch_first=True)

        self.logits = torch.nn.Linear(hidden_dim, vocab_dim)
        
    def forward(self, word_ix, latent_vector=None, h = None, c = None):

        if h is None and c is None:
            initial_cell = self.latent_to_h0(latent_vector)[None]
            initial_hid = self.latent_to_c0(latent_vector)[None]
        else:
           initial_cell = c
           initial_hid = h
        
        emb = self.emb(word_ix)

        lstm_out, (h, c) = self.lstm(emb, (initial_cell, initial_hid))

        return self.logits(lstm_out), (h, c)

In [ ]:
model = Encoder(vocab_dim = len(word_to_ind))

In [ ]:
model(x_b).shape

torch.Size([2, 1000])

In [45]:
decoder = Decoder(vocab_dim = 100)

In [46]:
latent = torch.randn(2, 1000)

In [47]:
x = torch.ones(2, 1).long()*word_to_ind.get('<START>', 1) # Да в словарь еще <START> токен нужно добавить

In [48]:
max_len = 10
pred = torch.zeros((len(x), max_len))
h_i = None
c_i = None
for i in range(max_len):
  p_i, (h_i, c_i) = decoder(x, latent_vector=x_b, h = h_i, c = c_i)
  pred[:, i] = torch.argmax(p_i[:, 0, :], dim=-1)

In [49]:
pred

tensor([[ 9., 90., 57., 57., 94., 94., 94., 94., 94., 94.],
        [74., 94., 94., 94., 94., 94., 94., 94., 94., 94.]])